In [529]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt  # to plot
import matplotlib as mpl 

from hardwareModel import HardwareModel
import networkx as nx

from scipy import optimize       # to compare

import seaborn as sns
sns.set(context="talk", style="darkgrid", palette="hls", font="sans-serif", font_scale=1.05)

FIGSIZE = (19, 8)  #: Figure size, in inches!
mpl.rcParams['figure.figsize'] = FIGSIZE
rng = rn.default_rng()

In [530]:
def annealing(random_start,  # get rid of function parameters
              cost_function,
              random_neighbour,
              acceptance,
              temperature,
              positions,
              edges,
              areas,
              maxsteps=10,
              debug=True):
    """ Optimize the black-box function 'cost_function' with the simulated annealing algorithm."""
    for i in range(len(positions)):
        state = init_positions()
        positions[i] = state
    
    wireLengths = wirelengths(positions, edges)
    #print(np.sum(list(wireLengths.values())))
    cost = cost_function(wireLengths, positions, areas)
    states, costs = [[], [], []], [cost]
    optimalPos = positions.copy()
    minWireLengths = float("inf")
    
    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)

        for i in range(len(positions)):
            new_state = random_neighbour(positions[i], fraction)
            positions[i] = new_state
            wireLengths = wirelengths(positions, edges)
            wireLengthSum = np.sum(list(wireLengths.values()))
            new_cost = cost_function(wireLengths, positions, areas)
            #if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(step, maxsteps, T, state, cost, new_state, new_cost))
            if acceptance_probability(cost, new_cost, T) > rn.random():
                state, cost = new_state, new_cost
            if wireLengthSum < minWireLengths and overlapCost(positions, areas) == 0:
                minWireLengths = wireLengthSum
                optimalPos = positions.copy()
                #print('wirelength:', minWireLengths, '|', 'pos:', optimalPos)
            
            states[i].append(state)
            costs.append(new_cost)
        #print("cost:", cost, "|", "new_cost:", new_cost)
        
        #print(np.sum(list(wireLengths.values())))
            # print("  ==> Accept it!")
        # else:
        #    print("  ==> Reject it...")

    print('wirelength:', minWireLengths, '|', 'overlap cost:', overlapCost(optimalPos, areas), '|', 'pos:', optimalPos)
        
    return state, cost_function(wireLengths, positions, areas), states, costs, minWireLengths, optimalPos

In [531]:
hw = HardwareModel(cfg='aladdin_const_with_mem')
digraph = hw.netlist
N = len(digraph.nodes)

netlist: [('Add0', {'type': 'pe', 'function': 'Add', 'in_use': 0, 'idx': 0}), ('Regs0', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 0}), ('Regs1', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 1}), ('Regs2', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 2}), ('Mult0', {'type': 'pe', 'function': 'Mult', 'in_use': 0, 'idx': 0}), ('Eq0', {'type': 'pe', 'function': 'Eq', 'in_use': 0, 'idx': 0}), ('Buf0', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 0, 'size': 22}), ('Buf1', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 1, 'size': 22}), ('Mem0', {'type': 'mem', 'function': 'MainMem', 'in_use': 0, 'idx': 0, 'size': 1024}), ('Regs3', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 3})]


In [532]:
interval = (-10, 10)
N = 3
#wirelengths = {(0, 1) : 2, (2, 3) : 2}
positions = [(0, 0), (0, 0), (2, 0)]
edges = [(0, 1), (0, 2)]
areas = [1, 1, 1]

def wirelengths(x, e):
    repeats = set()
    output = {}

    for edge in e:
        if (edge[1], edge[0]) not in repeats:
            repeats.add(edge)
            position0 = x[edge[0]]
            position1 = x[edge[1]]
            wirelength = ((position0[0] - position1[0]) ** 2 + (position0[1] - position1[1]) ** 2) ** 0.5
            output[edge] = wirelength

    return output

def f(x, positions, areas):
    """ Function to minimize."""
    #print(areas)
    #print("Sum of wirelengths:", np.sum(list(x.values())), "|", "Sum of overlaps:", overlapCost(positions, areas))
    return sum(x.values()) + overlapCost(positions, areas)

def overlap(first, sec, area1, area2):
    s1 = (area1 ** 0.5) / 2
    s2 = (area2 ** 0.5) / 2

    #print(first, sec)
    if (first[0] - s1 > sec[0] + s2) or (sec[0] - s2 > first[0] + s2) or (first[1] - s1 > sec[1] + s2) or (sec[1] - s2 > first[1] + s2):
        return 0
    
    deltaX = min([first[0] + s1, sec[0] + s2]) - max([first[0] - s1, sec[0] - s2])
    deltaY = min((first[1] + s1, sec[1] + s2)) - max((first[1] - s1, sec[1] - s2))
    #print('first square:', (first[0] + s1, first[0] - s1), '|', 'second square:', (sec[0] + s2, sec[0] - s2))

    #print('deltaX:', deltaX, '|', 'deltaY:', deltaY)
    if deltaX < 0 and deltaY < 0:
        return 0
    if deltaX == 0 and deltaY != 0:
        return abs(s1 * deltaY)
    if deltaX != 0 and deltaY == 0:
        return abs(s2 * deltaX)

    #print(deltaX * deltaY)
    
    return deltaX * deltaY

def overlapCost(pVectors, areas, const=1):
    totalCost = 0
    #print(areas)
    
    for i in range(len(pVectors) - 1):
        for j in range(i + 1, len(pVectors)):
            totalCost += overlap(pVectors[i], pVectors[j], areas[i], areas[j])
            #print(totalCost)

    return totalCost

def clip(x):
    """ Force x to be in the interval."""
    a, b = interval
    return max(min(x, b), a)

In [533]:
def random_start():
    """ Random point in the interval."""
    a, b = interval
    x = a + (b - a) * rn.random_sample()
    y = a + (b - a) * rn.random_sample()
    return (x, y)

def init_positions():
    return rng.normal(loc=0.0, scale=1.0, size=2)

In [534]:
def cost_function(x, positions, areas):
    """ Cost of x = f(x)."""
    #print(areas)
    return f(x, positions, areas)

In [535]:
def random_neighbour(x, fraction=1):
    """Move a little bit x, from the left or the right."""
    amplitude = (max(interval) - min(interval)) * (1 - fraction) / 10
    deltaX = (-amplitude/2.) + amplitude * rng.normal(loc=0.0, scale=1.0, size=None)
    deltaY = (-amplitude/2.) + amplitude * rng.normal(loc=0.0, scale=1.0, size=None)
    return (clip(x[0] + deltaX), clip(x[1] + deltaY))

In [536]:
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        # print("    - Acceptance probabilty = 1 as new_cost = {} < cost = {}...".format(new_cost, cost))
        return 1
    else:
        p = np.exp(- (new_cost - cost) / temperature)
        # print("    - Acceptance probabilty = {:.3g}...".format(p))
        return p

In [537]:
def temperature(fraction):
    """ Example of temperature dicreasing as the process goes on."""
    return max(0.01, min(1, 1 - fraction))

In [538]:
annealing(random_start, cost_function, random_neighbour, acceptance_probability, temperature, positions, edges, areas, maxsteps=30, debug=True);

wirelength: 3.277065528636518 | overlap cost: 0 | pos: [(0.48459156513231516, -0.8979920500468163), array([1.29374659, 0.37177896]), array([-0.86971739,  0.24379594])]


In [539]:
state, c, states, costs, minWireLength, optimalPos = annealing(random_start, cost_function, random_neighbour, acceptance_probability, temperature, positions, edges, areas, maxsteps=1000, debug=False)

print(optimalPos)
#print(overlapCost(optimalPos, areas))

wirelength: 2.3268328849112994 | overlap cost: 0 | pos: [(-7.927570651629971, -10), (-6.832963068792537, -10), (-9.159795953703837, -10)]
[(-7.927570651629971, -10), (-6.832963068792537, -10), (-9.159795953703837, -10)]


In [524]:
print(areas)

[1, 1, 1]


In [511]:
print(states[0])
print('\n')
print(states[1])
print('\n')
print(states[2])

[array([1.17012663, 0.01310997]), (-0.8153341749585028, -2.231842951068233), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (0.30126435906479043, -2.0608709477328566), (-7.949429034128319, -10), (-7.3128745030796125, -10), (-7.3128745030796125, -10), (-10, -7.500062516951203), (-10, -9.229827841357235), (-10, -8.365053055721422), (-10, -8.365053055721422), (-9.79583505759021, -10), (-9.482375668762728, -9.737560630085373), (-9.91455257930386, -9.700219609211244), (-9.957

In [424]:
print(len(states[0]))
print(len(states[1]))
print(len(states[2]))

1000
1000
1000
